<a href="https://colab.research.google.com/github/lupis30puc/bipm_text_analytics_exercises/blob/main/COVID_similarity_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import en_core_web_sm
import csv
import pickle
from google.colab import files

from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
from nltk.corpus import stopwords

from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from scipy.spatial.distance import cosine, cdist

## Loading Datasets

In [ ]:
# Loading the filtered original dataset
uploaded=files.upload()

Saving meta_stemmed to meta_stemmed


In [ ]:
smoke_= pickle.load(open('meta_stemmed', 'rb'))
smoke_.head()

,level_0,publish_time,title,abstract,date,year,language
0,4131,2020-01-09,Cocaine and amphetamine-regulated transcript p...,iolog cocain amphetamin regul transcript pre...,2020-01-09,2020,en
1,4195,2020-01-21,Pioneering Experience of Uniportal Video-Assis...,optim wai treat sev thorac scoliosi remain con...,2020-01-21,2020,en
2,4310,2020-02-10,Epidemiological and clinical characteristics o...,surveil iolog charactist car hai eldli...,2020-02-10,2020,en
3,4358,2020-02-07,Therapeutic strategies in an outbreak scenario...,n origin effort foc contain quaran...,2020-02-07,2020,en
4,4485,2020-01-17,Single-Virus Tracking: From Imaging Methodolog...,im text un mechan assembl crucial s treat ...,2020-01-17,2020,en


In [ ]:
for i in range(len(smoke_.abstract)):
        smoke_.abstract[i] = strip_multiple_whitespaces(str(smoke_.abstract[i])) #Remove multiple whitespaces
        smoke_.abstract[i] = strip_short(str(smoke_.abstract[i]), minsize=3) #Remove short words
        smoke_.abstract[i] = strip_numeric(str(smoke_.abstract[i])) #Remove digits
        smoke_.abstract[i] = strip_punctuation(str(smoke_.abstract[i]))  #Remove punctuation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [ ]:
# Making the stemmed abstracts into a list
smoke_data = list(smoke_.abstract)
smoke_data[0]

'iolog cocain amphetamin regul transcript prepropeptid cartpt gene polymorph modifi diet obes ionship intact cartpt gene polymorph diet qualiti indic investig intact major dietari indic diet qualiti intn dqi eat hei cartpt gene variant appar obes iranian cross section carri emploi appar obes adult bmi diet qualiti diet qualiti intn dqi eat hei item semi quantit valid food frequenc questionnair cartpt gene polymorph genotyp polymas chain reaction restrict frment length polymorph pcr–rflp techniqu blood concentr glycem mark lipid profil melanocyt stimul hormon msh outi peptid ana intact gene diet intact intact cartpt gene polymorph hei affect bmr intact sum glucos intact densiti lipoprotein cholestol hdl concentr intact adjt sex gene diet intact cartpt genotyp dqi fat mass intact waist circumf intact bodi mass bmi intact basal bol bmr intact sum fast glucos intact intact individu adjt confound intact cartpt genotyp adh hei dqi obes anthropometr bol factor'

In [ ]:
smoke_data[11190]

'factor intub extub design obsv set chico metropolitan area subject march april sociodemograph charactist intub prolong intub failur secondari intquartil rang male bodi mass intquartil rang intub eventu extub remain intub mean intub diabet intub logist sex oxygen satur histori diabet short breath factor intub bodi mass factor extub addit sign distress old male diabet requir intub intub old obes prolong intub otolaryngologist consult airwai man consid factor decis make'

In [ ]:
# Loading the target tables dataset
uploaded=files.upload()

Saving target_stemmed to target_stemmed


In [ ]:
target_df= pickle.load(open('target_stemmed', 'rb'))
target_df.head()

,Date,title,abstract
0,2020-05-22,Interim Analysis of Risk Factors for Severe Ou...,state grea death factor adult design surveil n...
1,2020-05-19,Factors Associated With Intubation and Prolong...,factor intub extub design obsv set chico metro...
2,2020-05-19,Risk factors for adverse clinical outcomes in ...,hetogen vari prognosi charactist cours off gui...
3,2020-05-16,Severe obesity is associated with higher in-ho...,york epicent undrepres minor obes appear affec...
4,2020-05-15,"Clinical Characteristics, Risk Factors and Out...",kuwait prior mass screen incom travel known su...


In [ ]:
for i in range(len(target_df.abstract)):
        target_df.abstract[i] = strip_multiple_whitespaces(str(target_df.abstract[i])) #Remove multiple whitespaces
        target_df.abstract[i] = strip_short(str(target_df.abstract[i]), minsize=3) #Remove short words
        target_df.abstract[i] = strip_numeric(str(target_df.abstract[i])) #Remove digits
        target_df.abstract[i] = strip_punctuation(str(target_df.abstract[i]))  #Remove punctuation

In [ ]:
# Making the steemed abstracts into a list
smoke_target = list(target_df.abstract)
smoke_target[10]

'itali european hit est death europ prospect correl death conduct intens luigi sacco milan itali charactist alis march admiss dep probabl death kaplan mei censor april cox proport hazard factor death intquartil rang male iqr dep probabl death admiss hazard ahr obes ahr death ahr protein ahr creatinin kinas ahr admiss fatal italian add evid notion old obes advanc factor death'

In [ ]:
smoke_target[1]

'factor intub extub design obsv set chico metropolitan area subject march april sociodemograph charactist intub prolong intub failur secondari intquartil rang male bodi mass intquartil rang intub eventu extub remain intub mean intub diabet intub logist sex oxygen satur histori diabet short breath factor intub bodi mass factor extub addit sign distress old male diabet requir intub intub old obes prolong intub otolaryngologist consult airwai man consid factor decis make'

In [ ]:
print('We have this ' + str(len(smoke_target)) + ' many abstracts from the target table')
print('We have this ' + str(len(smoke_data)) + ' many abstracts from the filtered orginal dataset')

We have this 42 many abstracts from the target table
We have this 27550 many abstracts from the filtered orginal dataset


In [ ]:
# splitting filtered data corpus
corpus_data= [doc.split() for doc in smoke_data]
print(corpus_data[0])

['iolog', 'cocain', 'amphetamin', 'regul', 'transcript', 'prepropeptid', 'cartpt', 'gene', 'polymorph', 'modifi', 'diet', 'obes', 'ionship', 'intact', 'cartpt', 'gene', 'polymorph', 'diet', 'qualiti', 'indic', 'investig', 'intact', 'major', 'dietari', 'indic', 'diet', 'qualiti', 'intn', 'dqi', 'eat', 'hei', 'cartpt', 'gene', 'variant', 'appar', 'obes', 'iranian', 'cross', 'section', 'carri', 'emploi', 'appar', 'obes', 'adult', 'bmi', 'diet', 'qualiti', 'diet', 'qualiti', 'intn', 'dqi', 'eat', 'hei', 'item', 'semi', 'quantit', 'valid', 'food', 'frequenc', 'questionnair', 'cartpt', 'gene', 'polymorph', 'genotyp', 'polymas', 'chain', 'reaction', 'restrict', 'frment', 'length', 'polymorph', 'pcr–rflp', 'techniqu', 'blood', 'concentr', 'glycem', 'mark', 'lipid', 'profil', 'melanocyt', 'stimul', 'hormon', 'msh', 'outi', 'peptid', 'ana', 'intact', 'gene', 'diet', 'intact', 'intact', 'cartpt', 'gene', 'polymorph', 'hei', 'affect', 'bmr', 'intact', 'sum', 'glucos', 'intact', 'densiti', 'lipopro

In [ ]:
# splitting target tables corpus
corpus_target= [doc.split() for doc in smoke_target]
print(corpus_target[0])

['state', 'grea', 'death', 'factor', 'adult', 'design', 'surveil', 'network', 'set', 'state', 'march', 'sex', 'race', 'ethnic', 'undli', 'condit', 'nineti', 'pcent', 'undli', 'condit', 'requir', 'intens', 'icu', 'admiss', 'invas', 'mechan', 'ventil', 'vasopressor', 'factor', 'icu', 'admiss', 'vsu', 'arr', 'respect', 'male', 'sex', 'arr', 'obes', 'arr', 'immunosuppress', 'arr', 'diabet', 'arr', 'factor', 'mortal', 'vsu', 'arr', 'respect', 'male', 'sex', 'arr', 'immunosuppress', 'arr', 'renal', 'arr', 'chronic', 'lung', 'arr', 'cardiovascular', 'arr', 'neurolog', 'disord', 'arr', 'diabet', 'arr', 'race', 'ethnic', 'icu', 'admiss', 'death', 'limit', 'limit', 'discharg', 'complet', 'chart', 'abstract', 'access', 'exclud', 'mortal', 'markedli', 'charact', 'pson', 'est', 'defin', 'target', 'stgi']


## *Creating the doc2vec model:*

In [ ]:
# initiaize the model building vocabulary with the abstracts from the filtered original dataset
data_sentences = [TaggedDocument(doc, [i]) for i, doc in enumerate(corpus_data)]
d2v_model = Doc2Vec(vector_size=20, min_count=5, workers=11, alpha=0.025, epochs=200)
d2v_model.build_vocab(data_sentences)
d2v_model.train(data_sentences, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)
d2v_model.random.seed(0)

Taking into account that the model would be applied to the target tables, we initialize the model with more epochs. This eventually would result into an improvement for the embeddings in both sides: initial model and target one. With this we expect the embeddings generated from the same document to be identified as similar by the model.

In [ ]:
# Saving the model trainee with the orginal dataset
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("my_doc2vec_model")
d2v_model.save(fname)
model = Doc2Vec.load(fname) 

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# Taking target 1 as an example in the filtered dataset
check = smoke_.title.str.contains(target_df.title[1])

print(smoke_[check])

       level_0 publish_time  ...  year language
11190    63128         2020  ...  2020       en

[1 rows x 7 columns]


In [ ]:
# Applying the model to the target datset
d2v_target = []
for i in range(len(corpus_target)):
  model.random.seed(0)
  d2v_target.append(model.infer_vector(corpus_target[i], epochs=200))

d2v_target[1]

array([ 2.3583543, -2.0364895, -1.2837102,  6.3063893,  0.9693353,
       -4.1885796, -1.6564652, -1.5281469,  2.8807964,  7.6072106,
       -3.0475526,  3.1550748,  2.9225712, -4.464131 ,  5.0232353,
       -2.1893983,  2.5326688, -2.9129293, -3.2553499, -3.36051  ],
      dtype=float32)

In [ ]:
#Cheking the embedding for the model equivalent to the target 1 
model[11190]

array([ 2.2503307, -1.8134667, -1.0323012,  5.413439 ,  1.5454991,
       -4.326559 , -1.4373482, -1.1812778,  2.6636133,  7.102266 ,
       -2.817789 ,  2.9696128,  2.2011912, -3.9919333,  5.068195 ,
       -1.7924573,  2.8122442, -2.8066201, -2.9646938, -3.5182066],
      dtype=float32)

In [ ]:
# Checking that the similarity identifies the document on the original dataset
similars = model.docvecs.most_similar(positive=[d2v_target[1]], topn=3)
print(similars)

[(11190, 0.9959909915924072), (26251, 0.8352000713348389), (8379, 0.8316121101379395)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
# confirming that the stemmed abstracts are identical
print(smoke_['title'][11190])
print(smoke_data[11190])
print(smoke_target[1])

Factors Associated With Intubation and Prolonged Intubation in Hospitalized Patients With COVID-19
factor intub extub design obsv set chico metropolitan area subject march april sociodemograph charactist intub prolong intub failur secondari intquartil rang male bodi mass intquartil rang intub eventu extub remain intub mean intub diabet intub logist sex oxygen satur histori diabet short breath factor intub bodi mass factor extub addit sign distress old male diabet requir intub intub old obes prolong intub otolaryngologist consult airwai man consid factor decis make
factor intub extub design obsv set chico metropolitan area subject march april sociodemograph charactist intub prolong intub failur secondari intquartil rang male bodi mass intquartil rang intub eventu extub remain intub mean intub diabet intub logist sex oxygen satur histori diabet short breath factor intub bodi mass factor extub addit sign distress old male diabet requir intub intub old obes prolong intub otolaryngologist c

In [ ]:
# checking further similar documents
for i in range(len(similars)):
  art = similars[i][0]
  print('art-' + str(art) + ': ' + smoke_.title[art])
  print(smoke_data[art])
  print('')

art-11190: Factors Associated With Intubation and Prolonged Intubation in Hospitalized Patients With COVID-19
factor intub extub design obsv set chico metropolitan area subject march april sociodemograph charactist intub prolong intub failur secondari intquartil rang male bodi mass intquartil rang intub eventu extub remain intub mean intub diabet intub logist sex oxygen satur histori diabet short breath factor intub bodi mass factor extub addit sign distress old male diabet requir intub intub old obes prolong intub otolaryngologist consult airwai man consid factor decis make

art-26251: Neck circumference as reliable predictor of mechanical ventilation support in adult inpatients with COVID‐19: A multicentric prospective evaluation
especi sev eldli subject cardiobol comorbid neck circumf strongli cardiobol adjt bodi mass bmi pform prospect investig invas mechan ventil imv adult inpati mati prospect consecut enrol adult dedic ward italian march april admiss biochem anthropometr bmi prim